In [1]:
from lightgbm import LGBMClassifier
import lightgbm as lgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
import pandas as pd

# Load and prepare data
df = pd.read_csv("burst_adma_with_cpm_multi_sensors_cls/burst_adma_with_cpm_multi_sensors999_cls_all.csv")

# Convert object columns to categorical
for col in df.select_dtypes(include="object"):
    df[col] = df[col].astype("category")

X = df.drop("cls_detected", axis=1)
y = df["cls_detected"]

# Show class labels
print("Unique classes:", y.unique())

# Train/test split with stratification
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, stratify=y, random_state=42
)

# Set up LightGBM model for multiclass
num_classes = y.nunique()

model = LGBMClassifier(
    objective="multiclass",
    num_class=num_classes,
    max_depth=10,
    n_estimators=1000
)

# Fit with early stopping callback
model.fit(
    X_train, y_train,
    eval_set=[(X_test, y_test)],
    eval_metric="multi_logloss",
    callbacks=[lgb.early_stopping(stopping_rounds=10)],
)

# Predict and evaluate
y_pred = model.predict(X_test)
print(classification_report(y_test, y_pred))


Unique classes: [0 1 2 3 4 5 6 7 8 9]
[LightGBM] [Warning] Categorical features with more bins than the configured maximum bin number found.
[LightGBM] [Warning] For categorical features, max_bin and max_bin_by_feature may be ignored with a large number of categories.
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.998528 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 22837
[LightGBM] [Info] Number of data points in the train set: 15915263, number of used features: 19
[LightGBM] [Info] Start training from score -1.774311
[LightGBM] [Info] Start training from score -6.903321
[LightGBM] [Info] Start training from score -8.009027
[LightGBM] [Info] Start training from score -2.219580
[LightGBM] [Info] Start training from score -1.833900
[LightGBM] [Info] Start training from score -2.193644
[LightGBM] [Info] Start training from score -2.086688
[LightGBM] [Info] Start training from score -2.096901
[Light

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.svm import LinearSVC
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import make_pipeline
from sklearn.metrics import classification_report, matthews_corrcoef

# Load data
df = pd.read_csv("burst_adma_with_cpm_multi_sensors_cls/burst_adma_with_cpm_multi_sensors999_cls_all.csv")

# Split features and label
X = df.drop("cls_detected", axis=1)
y = df["cls_detected"]

# Identify categorical columns
cat_cols = X.select_dtypes(include="object").columns.tolist()

# Define preprocessing (one-hot encode categorical features)
preprocessor = ColumnTransformer(
    transformers=[("cat", OneHotEncoder(handle_unknown="ignore"), cat_cols)],
    remainder="passthrough"  # numeric columns stay as is
)

# Create LinearSVC pipeline
svm_model = make_pipeline(
    preprocessor,
    LinearSVC(max_iter=10000, dual=False)  # dual=False is faster for large dense datasets
)

# Train/test split
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, stratify=y, random_state=42
)

# Fit
svm_model.fit(X_train, y_train)

# Predict
y_pred = svm_model.predict(X_test)

# Evaluate
print(classification_report(y_test, y_pred))
print("Matthews Correlation Coefficient (MCC):", round(matthews_corrcoef(y_test, y_pred), 4))


              precision    recall  f1-score   support

           0       0.31      0.60      0.41   1156814
           1       0.89      0.98      0.93      6851
           2       0.99      1.00      0.99      2268
           3       0.61      0.81      0.69    741116
           4       0.31      0.57      0.40   1089894
           5       0.28      0.15      0.19    760588
           6       0.33      0.15      0.21    846447
           7       0.30      0.29      0.29    837847
           8       0.43      0.00      0.00    624071
           9       0.30      0.02      0.03    754932

    accuracy                           0.35   6820828
   macro avg       0.48      0.46      0.42   6820828
weighted avg       0.35      0.35      0.30   6820828

Matthews Correlation Coefficient (MCC): 0.2542


In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report, matthews_corrcoef
from sklearn.feature_extraction import FeatureHasher
from sklearn.decomposition import TruncatedSVD
from scipy import sparse
from scipy.sparse import hstack
from scipy.stats import mode
import faiss

# Load data
df = pd.read_csv("burst_adma_with_cpm_multi_sensors_cls/burst_adma_with_cpm_multi_sensors999_cls_all.csv")

# Feature / label split
X = df.drop("cls_detected", axis=1)
y = df["cls_detected"].astype("category").cat.codes

# Identify categorical and numerical columns
cat_cols = X.select_dtypes(include="object").columns.tolist()
num_cols = [col for col in X.columns if col not in cat_cols]

# Fill missing values
X = X.fillna("missing")
X[num_cols] = X[num_cols].fillna(0)

# Hash categorical features to sparse matrix
X_cat_dicts = X[cat_cols].astype(str).to_dict(orient="records")
hasher = FeatureHasher(n_features=1024, input_type='dict')
X_cat_hashed_sparse = hasher.transform(X_cat_dicts)

# Scale numerical features
X_num_scaled = StandardScaler().fit_transform(X[num_cols]).astype("float32")
X_num_sparse = sparse.csr_matrix(X_num_scaled)

# Combine features
X_combined_sparse = hstack([X_cat_hashed_sparse, X_num_sparse]).tocsr()

# Print estimated memory if made dense
approx_gb = X_combined_sparse.shape[0] * X_combined_sparse.shape[1] * 4 / 1024**3
print(f"Estimated size if dense: {approx_gb:.2f} GB")

# Reduce dimensions to a manageable size
svd = TruncatedSVD(n_components=128, random_state=42)
X_reduced = svd.fit_transform(X_combined_sparse).astype("float32")

# Split
X_train, X_test, y_train, y_test = train_test_split(
    X_reduced, y, test_size=0.3, stratify=y, random_state=42
)

# FAISS
d = X_train.shape[1]
index = faiss.IndexFlatL2(d)
index.add(X_train)

# Search
k = 5
_, indices = index.search(X_test, k)

# Predict
y_pred = mode(y_train.to_numpy()[indices], axis=1).mode.flatten()

# Evaluation
print(classification_report(y_test, y_pred))
print("MCC:", round(matthews_corrcoef(y_test, y_pred), 4))


: 

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import make_pipeline
from sklearn.metrics import classification_report, matthews_corrcoef
from sklearn.preprocessing import OrdinalEncoder

from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OrdinalEncoder

# Load dataset
df = pd.read_csv("burst_adma_with_cpm_multi_sensors_cls/burst_adma_with_cpm_multi_sensors999_cls_all.csv")

# Separate features and target
X = df.drop("cls_detected", axis=1)
y = df["cls_detected"]

# Identify categorical columns
cat_cols = X.select_dtypes(include="object").columns.tolist()
num_cols = [col for col in X.columns if col not in cat_cols]

preprocessor = ColumnTransformer(
    transformers=[
        ("cat", make_pipeline(
            SimpleImputer(strategy="constant", fill_value="missing"),
            OrdinalEncoder(handle_unknown="use_encoded_value", unknown_value=-1)
        ), cat_cols),
        ("num", SimpleImputer(strategy="mean"), num_cols)
    ]
) 

# Random Forest model pipeline
rf_model = make_pipeline(
    preprocessor,
    RandomForestClassifier(
        n_estimators=100,           # Increase for better accuracy, decrease for speed
        max_depth=20,               # Set to None for full growth
        n_jobs=-1,                  # Use all CPU cores
        class_weight="balanced",   # Useful for imbalanced classes
        random_state=42
    )
)

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, stratify=y, random_state=42
)

# Fit model
rf_model.fit(X_train, y_train)

# Predict
y_pred = rf_model.predict(X_test)

# Evaluate
print(classification_report(y_test, y_pred))
print("Matthews Correlation Coefficient (MCC):", round(matthews_corrcoef(y_test, y_pred), 4))


              precision    recall  f1-score   support

           0       0.74      0.49      0.59   1156814
           1       0.33      0.98      0.50      6851
           2       1.00      1.00      1.00      2268
           3       0.29      0.93      0.44    741116
           4       0.64      0.57      0.60   1089894
           5       0.53      0.51      0.52    760588
           6       0.64      0.48      0.55    846447
           7       0.77      0.36      0.49    837847
           8       0.58      0.52      0.55    624071
           9       0.71      0.33      0.45    754932

    accuracy                           0.52   6820828
   macro avg       0.62      0.62      0.57   6820828
weighted avg       0.62      0.52      0.53   6820828

Matthews Correlation Coefficient (MCC): 0.4742


In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import HistGradientBoostingClassifier
from sklearn.metrics import classification_report, matthews_corrcoef

# Load dataset
df = pd.read_csv("burst_adma_with_cpm_multi_sensors_cls/burst_adma_with_cpm_multi_sensors999_cls_all.csv")

# Separate features and target
X = df.drop("cls_detected", axis=1)
y = df["cls_detected"]

# ❗ Convert all object columns to pandas 'category' dtype BEFORE train-test split
for col in X.select_dtypes(include="object").columns:
    X[col] = X[col].astype("category")
assert not any(X.dtypes == "object"), "Object columns remain!"

print(X.dtypes) 

# Fill missing in categorical columns with a special label
for col in X.select_dtypes(include="category").columns:
    X[col] = X[col].cat.add_categories(["__missing__"]).fillna("__missing__")

print(X.isna().sum())

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, stratify=y, random_state=42
)

# Define model
model = HistGradientBoostingClassifier(
    max_iter=100,
    max_depth=10,
    learning_rate=0.1,
    early_stopping=True,
    random_state=42
)
print("Final dtypes before training:\n", X_train.dtypes)
print("Sample row:\n", X_train.head(1).to_dict())

# Fit
model.fit(X_train, y_train)

# Predict
y_pred = model.predict(X_test)

# Evaluate
print(classification_report(y_test, y_pred))
print("Matthews Correlation Coefficient (MCC):", round(matthews_corrcoef(y_test, y_pred), 4))


timestep                  float64
cpm_id                   category
vehicle_id               category
detected_vehicle_id      category
object_type              category
detected_object_type     category
label_detected              int64
x_detected                float64
y_detected                float64
speed_detected            float64
heading_detected          float64
acceleration_detected     float64
is_self                     int64
sensor_id                category
sensor_type              category
sensor_range                int64
weather_conditions       category
visibility_range            int64
detection_confidence      float64
dtype: object
timestep                 0
cpm_id                   0
vehicle_id               0
detected_vehicle_id      0
object_type              0
detected_object_type     0
label_detected           0
x_detected               0
y_detected               0
speed_detected           0
heading_detected         0
acceleration_detected    0
is_self         

ValueError: could not convert string to float: 'CPM_truck43_586.0'

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import SGDClassifier
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import make_pipeline
from sklearn.metrics import classification_report, matthews_corrcoef

# Load data
df = pd.read_csv("burst_adma_with_cpm_multi_sensors_cls/burst_adma_with_cpm_multi_sensors999_cls_all.csv")

# Split features and label
X = df.drop("cls_detected", axis=1)
y = df["cls_detected"]

# Identify categorical columns
cat_cols = X.select_dtypes(include="object").columns.tolist()

# Define preprocessing (OneHotEncoding)
preprocessor = ColumnTransformer(
    transformers=[
        ("cat", OneHotEncoder(handle_unknown="ignore"), cat_cols)
    ],
    remainder="passthrough"
)

# Create SGDClassifier pipeline
sgd_model = make_pipeline(
    preprocessor,
    SGDClassifier(
        loss="hinge",        # Linear SVM
        max_iter=1000,
        tol=1e-3,
        class_weight="balanced",  # optional if class imbalance
        random_state=42,
        n_jobs=-1
    )
)

# Train/test split
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, stratify=y, random_state=42
)

# Fit
sgd_model.fit(X_train, y_train)

# Predict
y_pred = sgd_model.predict(X_test)

# Evaluate
print(classification_report(y_test, y_pred))
print("Matthews Correlation Coefficient (MCC):", round(matthews_corrcoef(y_test, y_pred), 4))


TypeError: cannot unpack non-iterable function object